In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
import tensorflow as tf
import matplotlib.pyplot as plt
# Data Visualization 
import plotly.express as px
import random
from tensorflow.keras import layers, Model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

ModuleNotFoundError: No module named 'tensorflow'

# Dataset Organization
The study employs a comprehensive dataset consisting of images of skin lesions, specifically focusing on identifying monkeypox infections. The dataset is meticulously organized into three distinct sets to facilitate the training, validation, and evaluation phases of the machine learning model. Each set is stored in a separate directory, ensuring a streamlined process for model development and testing. The directory paths for these datasets are as follows:

Training Set: The training set, essential for the initial training of the neural network, is located at /monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Train/. This dataset comprises a diverse collection of lesion images used to teach the model the characteristics of monkeypox infections.

Validation Set: The validation set, used to tune the model's hyperparameters and prevent overfitting, is found at /monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/. Periodic evaluation of the model on this dataset during training allows for adjustments before final testing, ensuring the model generalizes well to new data.

Test Set: The test set, crucial for evaluating the model's performance on unseen data, is stored at /monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Test/. This final evaluation provides an unbiased assessment of the model's ability to classify new images accurately, reflecting its potential effectiveness in real-world applications.

The division of the dataset into training, validation, and testing sets is a standard practice in machine learning and deep learning projects. It ensures that the model can learn from a variety of examples, adjust to prevent overfitting, and finally, be tested to provide an accurate measure of its performance. This structured approach to dataset organization is crucial for developing robust and reliable predictive models.

In [ ]:
train_dir = "D:/project_monkey/Project_Monkey/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Train"
val_dir = "D:/project_monkey/Project_Monkey/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val"
test_dir = "D:/project_monkey/Project_Monkey/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Test"

# Data Description and Preliminary Exploration

To facilitate a comprehensive analysis of the monkeypox skin lesion dataset, the study leverages a metadata file that contains detailed information about each image within the dataset. This metadata file, named Monkeypox_Dataset_metadata.csv, is pivotal for understanding the characteristics and annotations of the images. The acquisition of this metadata is accomplished through the utilization of the Pandas library, a powerful data manipulation tool in Python, which provides efficient and straightforward methods for handling structured data.

In [ ]:
info_file = 'D:/project_monkey/Project_Monkey/monkeypox-skin-lesion-dataset/Monkeypox_Dataset_metadata.csv'
info = pd.read_csv(info_file)
info.head()

# Data Preparation

Image Dataset Construction
To develop and evaluate the machine learning model for classifying skin lesions as indicative of monkeypox or not, the study employed high-resolution images from a structured dataset. The images were organized into separate directories for training, validation, and testing, facilitating a streamlined workflow for model development and evaluation. A crucial step in preparing these images for input into the neural network involved resizing and batching the images, operations that were performed using TensorFlow, a comprehensive open-source platform for machine learning.

In [ ]:
# The dimensions for resizing the images were standardized to 224 by 224 pixels, a common resolution for convolutional neural network (CNN)
# inputs due to its balance between detail retention and computational efficiency. This size standardization ensures that the input layer of the CNN receives uniformly sized images, which is critical for the learning process.
IMG_SIZE = (224, 224)

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=IMG_SIZE,
                                                                 label_mode="binary",
                                                                 batch_size=32)

test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                 image_size=IMG_SIZE,
                                                                 label_mode="binary",
                                                                 batch_size=32,
                                                                shuffle=False)

val_data = tf.keras.preprocessing.image_dataset_from_directory(directory=val_dir,
                                                                 image_size=IMG_SIZE,
                                                                 label_mode="binary",
                                                                 batch_size=32,
                                                                shuffle=False)

# Exploratory Data Analysis

Visualization of Class Distribution
A crucial step in the exploratory analysis of the dataset involved assessing the distribution of images across different classes. This assessment aimed to identify any imbalance in the dataset that could potentially bias the machine learning model's training and evaluation processes. To facilitate this analysis, a custom Python function, visualize_class_distribution, was developed and employed. This function automates the process of calculating and visualizing the number of images available for each class within the dataset.

In [ ]:
# Function to visualize class distribution
def visualize_class_distribution(data_dir):
    labels = os.listdir(data_dir)
    num_images_per_class = []

    for label in labels:
        label_dir = os.path.join(data_dir, label)
        num_images_per_class.append(len(os.listdir(label_dir)))

    # Plotting class distribution
    plt.figure(figsize=(10, 6))
    sns.barplot(x=labels, y=num_images_per_class)
    plt.title('Class Distribution')
    plt.xlabel('Class')
    plt.ylabel('Number of Images')
    plt.xticks(rotation=45)
    plt.show()

# Perform class distribution visualization
visualize_class_distribution(train_dir)

The visualize_class_distribution function takes a single parameter, data_dir, which specifies the directory path containing the dataset's image classes. Within this directory, each class's images are expected to be stored in separate subdirectories, named according to the class they represent. The function performs the following operations:

Class Identification: It identifies the available classes by listing the directories within data_dir. Each directory name is treated as a class label.

Image Counting: For each class, the function counts the number of images by listing the files within each class's corresponding subdirectory. This count represents the total number of images available for that class.

Data Visualization: Using Matplotlib and Seaborn libraries, the function then creates a bar plot to visualize the class distribution. The x-axis of the plot represents the class labels, while the y-axis indicates the number of images per class. Additional plot settings, such as figure size, title, axis labels, and x-axis tick rotation, are applied to enhance readability and presentation.

In [ ]:
#fig = px.pie(
#    names=info.Label.unique(),
#    values=info.Label.value_counts(),
#    hole=0.2,
#    width=500,
#    height=500
#)
#fig.update_layout({'title':{'text':'Class Distribution','x':0.45}})
#fig.show()

In [ ]:
# Function to display additional sample images
def display_additional_samples(data_dir, num_samples=5):
    labels = os.listdir(data_dir)

    plt.figure(figsize=(15, 10))
    for i, label in enumerate(labels):
        label_dir = os.path.join(data_dir, label)
        images = os.listdir(label_dir)[:num_samples]
        for j, image in enumerate(images):
            plt.subplot(len(labels), num_samples, i * num_samples + j + 1)
            img = plt.imread(os.path.join(label_dir, image))
            plt.imshow(img)
            plt.title(label)
            plt.axis('off')
    plt.show()

# Display additional sample images
display_additional_samples(train_dir)

**Methodology for Sample Visualization**
to enhance the understanding of the dataset's characteristics and ensure a comprehensive exploratory data analysis, a Python function, display_additional_samples, was developed and implemented. This function is designed to systematically display a specified number of sample images from each class present in the dataset. Such visualization aids in assessing the variability within and between classes, crucial for informing subsequent data preprocessing and model training strategies.

In [ ]:
# Visualizing data
def visualize_random_images(dataset_type="train", label_type="Others"):
    
    sample = 9
    
    plt.figure(figsize=(15, 8))
    type_dir = train_dir if dataset_type=="train" else test_dir
    base_dir = os.path.join(type_dir, label_type)
    images = random.sample(os.listdir(base_dir), 9)
    
    for i, image in enumerate(images):
        plt.subplot(3, 3, i+1)
        img = plt.imread(os.path.join(base_dir, image))
        plt.imshow(img)

In [ ]:
visualize_random_images("train", "Monkeypox")

In [ ]:
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

# Function Implementation
The plot_loss_curves function leverages the TensorFlow model's History object, which records performance metrics at successive epochs during the model's training process. The History object encapsulates detailed logs of loss and accuracy metrics, facilitating a comprehensive analysis of the model's training journey. The visualization process is delineated as follows:

Metric Extraction: The function extracts the history of loss (loss, val_loss) and accuracy (accuracy, val_accuracy) metrics from the provided History object. These metrics represent the model's performance on the training and validation datasets across all epochs.

Epoch Enumeration: The epochs during which the model was trained are enumerated, providing a temporal axis for the performance plots.

Loss Visualization: A plot is generated to visualize the training and validation loss over epochs. This plot elucidates the model's optimization trajectory, highlighting how closely the model learns to predict the training data while generalizing to unseen validation data.

Accuracy Visualization: In a separate plot, training and validation accuracy metrics are visualized over epochs. This plot showcases the model's improving prediction capabilities as training progresses, offering insights into the model's efficacy in correctly classifying the data.

The dual plots of loss and accuracy serve as fundamental indicators of the model's training and validation performance, enabling the identification of key training phases such as rapid learning, plateauing of performance, or the onset of overfitting.

In [ ]:
#The EfficientNetB3 model is initialized with the include_top=False parameter to exclude the original top layers (i.e., the fully connected layers) intended for classification. This exclusion allows for the adaptation of the model to our specific task requirements. 
base_model = tf.keras.applications.EfficientNetB3(include_top=False)
base_model.trainable = True

for layer in base_model.layers[:-5]:
  layer.trainable = False

#Input Layer: An input layer is defined to accommodate images of shape 224x224 pixels with 3 color channels, matching the input dimensions expected by EfficientNetB3.
inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
base_layer = base_model(inputs)
#Dropout Layer 1: A dropout layer with a rate of 0.5 follows, introduced to mitigate overfitting by randomly omitting a portion of feature detectors during training.
dropout_layer_1 = layers.Dropout(0.5)(base_layer)
#Flattening Layer: The feature maps are then flattened, transforming the 2D feature maps into a 1D vector suitable for the dense layers.
flat_layer = layers.Flatten()(dropout_layer_1)
#Dense Layer 1: A dense layer with 256 neurons and ReLU activation function is employed to learn high-level patterns from the flat feature vector.
dense_1 = layers.Dense(256, activation="relu")(flat_layer)
#Dropout Layer 2: Another dropout layer with a rate of 0.5 is applied after the first dense layer to further prevent overfitting.
dropout_layer_2 = layers.Dropout(0.5)(dense_1)
#Dense Layer 2: Subsequently, a second dense layer with 128 neurons and ReLU activation follows, aimed at refining the learned representations.
dense_2 = layers.Dense(128, activation="relu")(dropout_layer_2)
#Output Layer: The final layer is a dense layer with a single neuron and a sigmoid activation function, designed to output the probability of the image belonging to the target class.
outputs = layers.Dense(1, activation="sigmoid")(dense_2)
model = Model(inputs, outputs)

# Model Architecture
Incorporation of EfficientNetB3 as a Base Model
To harness the advantages of pre-trained models for our image classification task, the study employs the EfficientNetB3 architecture, sourced from TensorFlow's Keras applications module. EfficientNetB3 is renowned for its efficiency and effectiveness in balancing model depth, width, and resolution, making it a prime candidate for serving as the foundational layer of our customized neural network.

In [ ]:
#Optimizer: The Adam optimizer was chosen due to its effectiveness in handling sparse gradients and its adaptability in adjusting the learning rate during training. The initial learning rate was set to 0.001, a value commonly used as a starting point for many types of neural networks.
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

#Model Fitting
#Following the model's compilation, it underwent a training process using the training dataset (train_data) with validation performed on a separate dataset (val_data). The training was conducted over 20 epochs, allowing the model to iteratively learn from the training data while its performance was simultaneously evaluated on the validation data to monitor for overfitting.
history = model.fit(train_data,
                    epochs=20,
                    validation_data=val_data)

# Model Training and Optimization
Compilation of the Neural Network Model
prior to training, the neural network model was compiled with specific configurations to optimize its performance for the binary classification task. The compilation process involved the selection of an optimizer, a loss function, and metrics for evaluating the model's performance.

In [ ]:
plot_loss_curves(history)

In [ ]:
model.evaluate(test_data)

In [ ]:
y_pred = tf.math.round(model.predict(test_data))
y_true = []
for images, labels in test_data.unbatch():
  y_true.append(labels.numpy())

In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
print(classification_report(y_true, y_pred, target_names=train_data.class_names))

In [ ]:
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm.astype("int"), annot=True)

In [ ]:
# สร้างแบบจำลองใหม่โดยใช้ ResNet50
#base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
#base_model.trainable = True  # ทำให้ทุกชั้นของ ResNet50 สามารถฝึกได้

# สร้างสถาปัตยกรรมของแบบจำลอง
#inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
#x = base_model(inputs, training=True)
#x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
#x = layers.Dense(256, activation="relu")(x)
#x = layers.Dropout(0.5)(x)
#outputs = layers.Dense(1, activation="sigmoid")(x)
#model_resnet50 = Model(inputs, outputs)

# คอมไพล์แบบจำลอง
#model_resnet50.compile(optimizer=tf.keras.optimizers.Adam(),
#                      loss='binary_crossentropy',
#                       metrics=['accuracy'])

# ฝึกแบบจำลอง
#history_resnet50 = model_resnet50.fit(train_data,
#                                      epochs=20,
#                                      validation_data=val_data)


In [ ]:
#def plot_loss_curves(history):
#    """
#    Returns separate loss curves for training and validation metrics.
#    """
#    loss = history.history['loss']
#    val_loss = history.history['val_loss']

#    accuracy = history.history['accuracy']
#    val_accuracy = history.history['val_accuracy']

#    epochs = range(len(history.history['loss']))

    # Plot loss
#    plt.figure(figsize=(8, 8))
#    plt.plot(epochs, loss, label='Training Loss')
#    plt.plot(epochs, val_loss, label='Validation Loss')
#    plt.title('Training and Validation Loss')
#    plt.xlabel('Epochs')
#    plt.legend()

    # Plot accuracy
#    plt.figure(figsize=(8, 8))
#    plt.plot(epochs, accuracy, label='Training Accuracy')
#    plt.plot(epochs, val_accuracy, label='Validation Accuracy')
#    plt.title('Training and Validation Accuracy')
#    plt.xlabel('Epochs')
#    plt.legend()

#plot_loss_curves(history_resnet50)

In [ ]:
# ประเมินโมเดลบนชุดข้อมูลทดสอบ
#test_loss, test_accuracy = model_resnet50.evaluate(test_data)
#print(f"Test Loss: {test_loss}")
#print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# ทำนายชุดข้อมูลทดสอบ
#y_pred = model_resnet50.predict(test_data)
#y_pred = tf.math.round(y_pred)  # ปรับให้เป็น 0 หรือ 1

# รวบรวม labels จริงจากชุดข้อมูลทดสอบ
#y_true = np.concatenate([labels for _, labels in test_data], axis=0)

# คำนวณความแม่นยำ
#accuracy = accuracy_score(y_true, y_pred)
#print(f"Accuracy: {accuracy}")

In [ ]:
# สร้างและแสดง confusion matrix
#cm = confusion_matrix(y_true, y_pred)
#plt.figure(figsize=(10, 7))
#sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')
#plt.xlabel('Predicted')
#plt.ylabel('True')
#plt.title('Confusion Matrix')

# Other modules for performance comparison testing

In [ ]:
from tensorflow.keras import layers, models

# สร้างสถาปัตยกรรมของแบบจำลอง
inputs = layers.Input(shape=(224, 224, 3), name="input_layer")

# Convolutional layer ชั้นที่ 1
x = layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Convolutional layer ชั้นที่ 2
x = layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Flatten the output layer to 1 dimension
x = layers.Flatten()(x)

# Add a fully connected layer with 256 hidden units and ReLU activation
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)  # Add a dropout rate of 0.5

# Add a final sigmoid layer for classification
outputs = layers.Dense(1, activation="sigmoid")(x)

# Create the model
model_custom = models.Model(inputs, outputs)

# Model Architecture
To address the specific requirements of our binary classification task, a custom convolutional neural network (CNN) model was designed and implemented. The architecture of this model was constructed to effectively extract and learn features from images, facilitating accurate classification. The model architecture comprises several layers, each serving a distinct purpose in the feature extraction and classification process:

**Input Layer**: The model begins with an input layer designed to receive images of shape 224x224 pixels with 3 color channels. This standardization is crucial for processing images in a uniform manner.

**First Convolutional Layer**: The initial layer of convolution employs 32 filters with a kernel size of 3x3, followed by a ReLU activation function. This layer is aimed at capturing basic patterns such as edges and colors. A subsequent max pooling layer with a pool size of 2x2 reduces the dimensionality, focusing on the most prominent features.

**Second Convolutional Layer**: A second convolutional layer, consisting of 64 filters and a 3x3 kernel, further processes the feature maps from the previous layer. This is again followed by max pooling, enhancing the model's ability to identify more complex features in the images.

**Flattening Layer**: The feature maps are flattened into a one-dimensional vector, preparing the data for the fully connected layers.

**Fully Connected Layer**: A dense layer with 256 neurons and ReLU activation function is employed to learn high-level patterns from the flattened feature maps. A dropout layer with a rate of 0.5 is incorporated to reduce the risk of overfitting by randomly omitting a portion of the neurons during training.

**Output Layer**: The final layer is a dense layer with a single neuron and a sigmoid activation function, designed to output the probability that the image belongs to the target class.

In [ ]:
#Optimizer: The Adam optimizer was employed with a learning rate of 0.001. Adam is renowned for its efficiency in gradient descent optimization by adaptively adjusting the learning rate, making it particularly suited for deep learning models where the landscape of the loss function can be highly complex.
#Loss Function: Binary crossentropy was chosen as the loss function. This selection is aligned with the binary nature of our classification task, where the model predicts the likelihood of images belonging to one of two possible classes. Binary crossentropy effectively quantifies the difference between the predicted probabilities and the actual binary labels, guiding the model towards better accuracy.
model_custom.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

#Model Training
#The model was trained over 20 epochs, allowing sufficient iterations for the model to learn from the training data progressively. An epoch in this context refers to one complete pass of the training dataset through the model.
history_custom = model_custom.fit(train_data,
                                  epochs=20,
                                  validation_data=val_data)


In [ ]:
def plot_loss_curves(history):
    """
    Returns separate loss curves for training and validation metrics.
    """
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    
    epochs = range(len(history.history['loss']))
    
    # Plot loss
    plt.figure(figsize=(8, 8))
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.legend()
    
    # Plot accuracy
    plt.figure(figsize=(8, 8))
    plt.plot(epochs, accuracy, label='Training Accuracy')
    plt.plot(epochs, val_accuracy, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.legend()

plot_loss_curves(history_custom)


# Evaluation of Model Training Dynamics
Visual Analysis of Training and Validation Metrics
To comprehensively assess the performance of the custom convolutional neural network model throughout its training phase, a visualization function, plot_loss_curves, was developed and applied. This function is instrumental in depicting the model's learning progression, facilitating the identification of key training characteristics such as convergence behavior, overfitting, or underfitting.

**Function Overview**
The plot_loss_curves function extracts the training and validation loss and accuracy metrics from the model's training history, provided by TensorFlow's History object. It then generates two separate plots:

**Training and Validation Loss**: The first plot visualizes the loss metrics, with the training loss depicted alongside the validation loss over the course of training epochs. This plot is crucial for observing how the model's performance improves and whether it begins to overfit to the training data by exhibiting an increasing trend in validation loss.

**Training and Validation Accuracy**: The second plot displays the accuracy metrics, plotting both training and validation accuracy. This visualization offers insights into the model's capability to correctly classify the images, highlighting the progression of accuracy and its stabilization or potential divergence, indicative of overfitting or underfitting, respectively.

**Visualization Process**
Upon invocation, plot_loss_curves executes the following steps:

**Metric Extraction**: The function retrieves the loss and accuracy metrics from the History object's history attribute, encompassing both training and validation phases.

**Epoch Enumeration**: It enumerates the epochs based on the length of the loss history, providing a temporal axis for the plots.

**Plot Configuration**: For each metric (loss and accuracy), a separate plot is configured, setting the figure size to 8x8 inches for clarity. The training and validation metrics are plotted as distinct lines, with appropriate labels and legends for differentiation.

**Display**: The plots are titled 'Training and Validation Loss' and 'Training and Validation Accuracy' respectively, with epochs on the x-axis and the metric values on the y-axis. Legends distinguish between training and validation lines, aiding in interpretation.

In [ ]:
# ประเมินโมเดลบนชุดข้อมูลทดสอบ
test_loss, test_accuracy = model_custom.evaluate(test_data)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
# ทำนายชุดข้อมูลทดสอบและปรับค่าเป็น 0 หรือ 1
y_pred = model_custom.predict(test_data)
y_pred_rounded = tf.round(y_pred)

# รวบรวม labels จริงจากชุดข้อมูลทดสอบ
y_true = np.concatenate([y for x, y in test_data], axis=0)

# คำนวณความแม่นยำ
accuracy = accuracy_score(y_true, y_pred_rounded)
print(f"Accuracy: {accuracy}")

# สร้างและแสดง confusion matrix
cm = confusion_matrix(y_true, y_pred_rounded)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')


# Conclusion

This project has attempted to apply deep learning technology to create models for classification and detection of monkeypox skin lesions using the "Monkeypox Skin Lesion Dataset" from Kaggle. General Convolutional Neural Network (CNN) and EfficientNetB3 were employed to compare the performance of both models in image classification. The results revealed that due to the nature of this image dataset requiring high accuracy in classification, the more complex and suitable model, EfficientNetB3, provided higher accuracy (approximately 89%). Although the standard CNN model could process data faster, it yielded lower accuracy (approximately 60%) for tasks demanding high precision. However, each model is inherently suitable for specific types of datasets, depending on the complexity of the data and the desired outcomes from that dataset.